In [1]:
% pip install jarvis-md

     |████████████████████████████████| 81kB 2.2MB/s 
     |████████████████████████████████| 645kB 10.8MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
import os, numpy as np, pandas as pd
from tensorflow import losses, optimizers
from tensorflow.keras import Input, Model, models, layers
from jarvis.train import datasets, custom

In [3]:
datasets.download(name='mr/brats-2020-096')

[ 2021-06-08 00:27:03 ] [====================] 100.000% : Extracting archive (0000486 / 0000486) 

{'code': '/data/raw/mr_brats_2020', 'data': '/data/raw/mr_brats_2020'}

# Global Regression Model

In [4]:
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-096', keyword='096*glb-org')

In [59]:
kwargs = {
    'kernel_size': (3, 3, 3),
    'padding': 'same'}

# --- Define lambda functions
conv = lambda x, filters, strides : layers.Conv3D(filters=filters, strides=strides, **kwargs)(x)
norm = lambda x : layers.BatchNormalization()(x)
relu = lambda x : layers.ReLU()(x)
tran = lambda x, filters, strides : layers.Conv3DTranspose(filters=filters, strides=strides, **kwargs)(x)

# --- Define stride-1, stride-2 blocks
conv1 = lambda filters, x : relu(norm(conv(x, filters, strides=1)))
conv2 = lambda filters, x : relu(norm(conv(x, filters, strides=2)))
tran2 = lambda filters, x : relu(norm(tran(x, filters, strides=2)))

In [6]:
inputs = client.get_inputs(Input)

In [7]:
# Contracting layers
l1 = conv1(16, inputs['dat'])
l2 = conv1(24, conv2(24, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))

f0 = layers.Flatten()(l5)

# --- Create logits
logits = {}
logits['survival'] = layers.Dense(1, activation = 'sigmoid', name='survival')(f0)

# --- Create model
model = Model(inputs=inputs, outputs=logits) 

In [8]:
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'survival': losses.MeanSquaredError()},
    experimental_run_tf_function=False)

In [9]:
# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=125, 
    epochs=8,
    validation_data=gen_valid,
    validation_steps=125,
    validation_freq=4,
    use_multiprocessing=True)

In [10]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

preds = []
trues = []
mae = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds.append(logits.ravel())
    trues.append(y['survival'].ravel())
    mae.append(np.abs(preds[-1] - trues[-1]))

preds = np.array(preds).ravel()
trues = np.array(trues).ravel()
mae = np.array(mae).ravel()

In [11]:
# --- Define columns
df = pd.DataFrame(index=np.arange(mae.size))
df['MAE'] = mae

# --- Print accuracy
print(df['MAE'].median())

In [12]:
model.save('./rshafi_model_1.hdf5')

# Standard Autoencoder Model

In [60]:
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-096', keyword='096*glb-org')

In [61]:
inputs = client.get_inputs(Input)

In [62]:
# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
 
# --- Define expanding layers
l6  = tran2(48, l5)
l7  = tran2(32, conv1(48, l6))
l8  = tran2(16, conv1(32, l7))
l9  = tran2(8,  conv1(16, l8))
l10 = conv1(8,  l9)

In [63]:
logits = {'recon': layers.Conv3D(filters=4, name='recon', **kwargs)(l10)}
ae = Model(inputs=inputs, outputs=logits)
 
# --- Create encoder
encoder = Model(inputs=inputs, outputs=l5)

In [64]:
def ae_generator(G):
    """
    Method to modify standard generator for autoencoder
    
    """
    for xs, ys in G:

        ys = {'recon': xs['dat']}

        yield xs, ys

In [65]:
# --- Compile model
ae.compile(
    optimizer=optimizers.Adam(learning_rate=1e-3),
    loss={'recon': losses.MeanSquaredError()},
    experimental_run_tf_function=False)

In [ ]:
# --- Train model
ae.fit(
    x=ae_generator(gen_train), 
    steps_per_epoch=250, 
    epochs=4,
    use_multiprocessing=True)

In [67]:
# --- Freeze encoder model weights
encoder.trainable = False

In [68]:
# --- Re-use the encoder model layers on input
latent = encoder(inputs)

In [69]:
# --- Finalize model
h0 = layers.Flatten()(latent)
h1 = layers.Dense(32, activation='relu')(h0)

logits = {}
logits['survival'] = layers.Dense(1, activation='sigmoid', name='survival')(h1)

# --- Create encoder
model = Model(inputs=inputs, outputs=logits)

In [70]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={'survival': losses.MeanSquaredError()},
    experimental_run_tf_function=False)

In [ ]:
# --- Train model
model.fit(
    x=gen_train, 
    steps_per_epoch=125, 
    epochs=4,
    validation_data=gen_valid,
    validation_steps=125,
    validation_freq=2,
    use_multiprocessing=True)

In [ ]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

preds = []
trues = []
mae = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds.append(logits.ravel())
    trues.append(y['survival'].ravel())
    mae.append(np.abs(preds[-1] - trues[-1]))

preds = np.array(preds).ravel()
trues = np.array(trues).ravel()
mae = np.array(mae).ravel()

In [ ]:
# --- Define columns
df = pd.DataFrame(index=np.arange(mae.size))
df['MAE'] = mae

# --- Print accuracy
print(df['MAE'].median())

In [74]:
model.save('./rshafi_model_2.hdf5')

#Fully-convolutional Network



In [28]:
# --- Create custom configs dict
configs = {'specs': {'ys': {
    'tumor': {
        'dtype': 'uint8',
        'loads': 'lbl-crp',
        'norms': {'clip': {'min': 0, 'max': 1}},
        'shape': [96, 96, 96, 1]},
    'survival': {
        'dtype': 'float32',
        'loads': 'survival_days_norm',
        'shape': [1]}}}}

# --- Create generators
gen_train, gen_valid, client = datasets.prepare(
    name='mr/brats-2020-096', 
    keyword='096*vox-org',
    configs=configs)

In [29]:
inputs = client.get_inputs(Input)

In [30]:
# --- Define contracting layers
l1 = conv1(8, inputs['dat'])
l2 = conv1(16, conv2(16, l1))
l3 = conv1(32, conv2(32, l2))
l4 = conv1(48, conv2(48, l3))
l5 = conv1(64, conv2(64, l4))
 
# --- Define expanding layers
l6  = tran2(48, l5)
l7  = tran2(32, conv1(48,l6))
l8  = tran2(16, conv1(32, l7))
l9  = tran2(8,  conv1(16, l8))
l10 = conv1(8,  l9)

In [31]:
# --- Create model
logits = {}
logits['survival'] = layers.Conv3D(filters=1, name='survival', activation='sigmoid', **kwargs)(l10)
logits['tumor'] = layers.Conv3D(filters=2, name='tumor', **kwargs)(l10)
model = Model(inputs=inputs, outputs=logits)

In [32]:
def generator(G):
    """
    Method to modify combined generator 
 
    """
    for xs, ys in G:
 
        # --- Reshape survival/tumor to 5D tensor
        survival = ys['survival'].reshape(-1, 1, 1, 1, 1)
        tumor = ys['tumor'].reshape(-1, 1, 1, 1, 1)
 
        # --- Replace ys['survival']/ys['tumor']
        ys['survival'] = np.zeros((survival.shape[0], 96, 96, 96, 1), dtype='float32')
        ys['survival'][:] = survival
        ys['tumor'] = np.zeros((survival.shape[0], 96, 96, 96, 1), dtype='float32')
        ys['tumor'][:] = survival
 
        yield xs, ys

In [33]:
# --- Compile model
model.compile(
    optimizer=optimizers.Adam(learning_rate=2e-4),
    loss={
        'survival': losses.MeanSquaredError(),
        'tumor': losses.SparseCategoricalCrossentropy(from_logits=True)},
    experimental_run_tf_function=False)

In [34]:
model.fit(x=generator(gen_train),
          steps_per_epoch=125, 
          epochs=8,
          use_multiprocessing=True)

In [35]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

preds = []
trues = []
mae = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds.append(logits.ravel())
    trues.append(y['survival'].ravel())
    mae.append(np.abs(preds[-1] - trues[-1]))

preds = np.array(preds).ravel()
trues = np.array(trues).ravel()
mae = np.array(mae).ravel()

In [36]:
# --- Define columns
df = pd.DataFrame(index=np.arange(mae.size))
df['MAE'] = mae

# --- Print accuracy
print(df['MAE'].median())

In [37]:
model.save('./rshafi_model_3.hdf5')

#Evaluation

##Model 1

In [39]:
model = models.load_model('./rshafi_model_1.hdf5')

In [40]:
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-096', keyword='096*glb-org')

In [41]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

preds = []
trues = []
mae = []
validation_size = 0
training_size = 0
for x, y in test_valid:

    validation_size += 1
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds.append(logits.ravel())
    trues.append(y['survival'].ravel())
    mae.append(np.abs(preds[-1] - trues[-1]))

preds = np.array(preds).ravel()
trues = np.array(trues).ravel()
mae = np.array(mae).ravel()

preds_train = []
trues_train = []
mae_train = []

for x, y in test_train:
    
    training_size += 1
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds_train.append(logits.ravel())
    trues_train.append(y['survival'].ravel())
    mae_train.append(np.abs(preds_train[-1] - trues_train[-1]))

preds_train = np.array(preds_train).ravel()
trues_train = np.array(trues_train).ravel()
mae_train = np.array(mae_train).ravel()

[ 2021-06-08 00:29:09 ] [====================] 100.000% : Iterating | 000185    

In [42]:
# --- Define columns
df = pd.DataFrame(index=np.arange(mae.size))
df['MAE'] = mae
print("Size of validation dataset: ", validation_size)
# --- Print accuracy (validation)
print(df['MAE'].median())
print(df['MAE'].mean())
print(df['MAE'].quantile(0.25))
print(df['MAE'].quantile(0.75))
print("Size of training dataset: ", training_size)
# --- Print accuracy (training)
df = pd.DataFrame(index=np.arange(mae_train.size))
df['MAET'] = mae_train
print(df['MAET'].median())
print(df['MAET'].mean())
print(df['MAET'].quantile(0.25))
print(df['MAET'].quantile(0.75))

Size of validation dataset:  50
0.06436899304389954
0.08401628583669662
0.028337106108665466
0.11981521546840668
Size of training dataset:  185
0.07638853788375854
0.08996275812387466
0.052431344985961914
0.10954666137695312


In [43]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dat (InputLayer)             [(None, 96, 96, 96, 4)]   0         
_________________________________________________________________
conv3d (Conv3D)              (None, 96, 96, 96, 16)    1744      
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 96, 16)    64        
_________________________________________________________________
re_lu (ReLU)                 (None, 96, 96, 96, 16)    0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 48, 48, 48, 24)    10392     
_________________________________________________________________
batch_normalization_1 (Batch (None, 48, 48, 48, 24)    96        
_________________________________________________________________
re_lu_1 (ReLU)               (None, 48, 48, 48, 24)    0     

##Model 2

In [44]:
model = models.load_model('./rshafi_model_2.hdf5')

In [45]:
gen_train, gen_valid, client = datasets.prepare(name='mr/brats-2020-096', keyword='096*glb-org')

In [46]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

preds = []
trues = []
mae = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds.append(logits.ravel())
    trues.append(y['survival'].ravel())
    mae.append(np.abs(preds[-1] - trues[-1]))
    
preds = np.array(preds).ravel()
trues = np.array(trues).ravel()
mae = np.array(mae).ravel()

preds_train = []
trues_train = []
mae_train = []

for x, y in test_train:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds_train.append(logits.ravel())
    trues_train.append(y['survival'].ravel())
    mae_train.append(np.abs(preds_train[-1] - trues_train[-1]))

preds_train = np.array(preds_train).ravel()
trues_train = np.array(trues_train).ravel()
mae_train = np.array(mae_train).ravel()

[ 2021-06-08 00:29:42 ] [====================] 100.000% : Iterating | 000185    

In [47]:
# --- Define columns
df = pd.DataFrame(index=np.arange(mae.size))
df['MAE'] = mae
print("Size of validation dataset: ", validation_size)
# --- Print accuracy (validation)
print(df['MAE'].median())
print(df['MAE'].mean())
print(df['MAE'].quantile(0.25))
print(df['MAE'].quantile(0.75))
print("Size of training dataset: ", training_size)
# --- Print accuracy (training)
df = pd.DataFrame(index=np.arange(mae_train.size))
df['MAET'] = mae_train
print(df['MAET'].median())
print(df['MAET'].mean())
print(df['MAET'].quantile(0.25))
print(df['MAET'].quantile(0.75))

Size of validation dataset:  50
0.051491230726242065
0.05621673911809921
0.025715917348861694
0.07968433201313019
Size of training dataset:  185
0.020265579223632812
0.030029529705643654
0.010300695896148682
0.03207427263259888


In [48]:
model.summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dat (InputLayer)             [(None, 96, 96, 96, 4)]   0         
_________________________________________________________________
model_1 (Functional)         (None, 6, 6, 6, 64)       351560    
_________________________________________________________________
flatten (Flatten)            (None, 13824)             0         
_________________________________________________________________
dense (Dense)                (None, 32)                442400    
_________________________________________________________________
survival (Dense)             (None, 1)                 33        
Total params: 793,993
Trainable params: 442,433
Non-trainable params: 351,560
_________________________________________________________________


##Model 3

In [49]:
model = models.load_model('./rshafi_model_3.hdf5')

In [50]:
# --- Create custom configs dict
configs = {'specs': {'ys': {
    'tumor': {
        'dtype': 'uint8',
        'loads': 'lbl-crp',
        'norms': {'clip': {'min': 0, 'max': 1}},
        'shape': [96, 96, 96, 1]},
    'survival': {
        'dtype': 'float32',
        'loads': 'survival_days_norm',
        'shape': [1]}}}}

# --- Create generators
gen_train, gen_valid, client = datasets.prepare(
    name='mr/brats-2020-096', 
    keyword='096*vox-org',
    configs=configs)

In [51]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True, expand=True)

preds = []
trues = []
mae = []

for x, y in test_valid:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds.append(logits.ravel())
    trues.append(y['survival'].ravel())
    mae.append(np.abs(preds[-1] - trues[-1]))

preds = np.array(preds).ravel()
trues = np.array(trues).ravel()
mae = np.array(mae).ravel()

preds_train = []
trues_train = []
mae_train = []

for x, y in test_train:
    
    # --- Predict
    logits = model.predict(x['dat'])

    if type(logits) is dict:
        logits = logits['survival']

    # --- Aggregate
    preds_train.append(logits.ravel())
    trues_train.append(y['survival'].ravel())
    mae_train.append(np.abs(preds_train[-1] - trues_train[-1]))

preds_train = np.array(preds_train).ravel()
trues_train = np.array(trues_train).ravel()
mae_train = np.array(mae_train).ravel()

[ 2021-06-08 00:30:23 ] [====================] 100.000% : Iterating | 000185    

In [52]:
# --- Define columns
df = pd.DataFrame(index=np.arange(mae.size))
df['MAE'] = mae
print("Size of validation dataset: ", validation_size)
# --- Print accuracy (validation)
print(df['MAE'].median())
print(df['MAE'].mean())
print(df['MAE'].quantile(0.25))
print(df['MAE'].quantile(0.75))
print("Size of training dataset: ", training_size)
# --- Print accuracy (training)
df = pd.DataFrame(index=np.arange(mae_train.size))
df['MAET'] = mae_train
print(df['MAET'].median())
print(df['MAET'].mean())
print(df['MAET'].quantile(0.25))
print(df['MAET'].quantile(0.75))

Size of validation dataset:  50
0.049898386001586914
0.05688264220952988
0.026019632816314697
0.08414971828460693
Size of training dataset:  185
0.057033658027648926
0.03419819846749306
0.022500336170196533
0.1128864586353302


In [53]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
dat (InputLayer)                [(None, 96, 96, 96,  0                                            
__________________________________________________________________________________________________
conv3d (Conv3D)                 (None, 96, 96, 96, 8 872         dat[0][0]                        
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 96, 96, 96, 8 32          conv3d[0][0]                     
__________________________________________________________________________________________________
re_lu (ReLU)                    (None, 96, 96, 96, 8 0           batch_normalization[0][0]        
______________________________________________________________________________________________